## Download dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip 'drive/MyDrive/emg_imu_data.zip'


In [2]:
# 导入包
from c2net.context import prepare, upload_output
# 初始化导入数据集和预训练模型到容器内
c2net_context = prepare()



       ___                _            _            _          
      |__ \              | |          | |          | |         
  ___    ) | _ __    ___ | |_  ______ | |__    ___ | |_   __ _ 
 / __|  / / | '_ \  / _ \| __||______|| '_ \  / _ \| __| / _` |
| (__  / /_ | | | ||  __/| |_         | |_) ||  __/| |_ | (_| |
 \___||____||_| |_| \___| \__|        |_.__/  \___| \__| \__,_|
                                                               
         

If you have any problems while preparing the data, you can submit an issue in this repository: https://openi.pcl.ac.cn/OpenIOSSG/c2net-pypi
        
Detected .code_cache_file already exists, code has been prepared!
Start preparing the dataset emg_data ...
✅ Completed preparing the dataset emg_data
Start preparing the dataset imu_data ...
✅ Completed preparing the dataset imu_data
please set c2net_context.output_path as the output location


In [3]:
datasetPath = c2net_context.dataset_path
datasetPath

'/tmp/dataset'

## 配置环境

In [ ]:
# !pip install scikit-learn
!pip install scikit-learn -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

## Fix Random Seed

In [4]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import os

class EMGDataset(Dataset):
    def __init__(self, directory, seq_length=790, scaler=None):
        self.directory = directory
        self.seq_length = seq_length
        self.files = []
        self.labels = []
        self.scaler = scaler  # 将预先计算好的Scaler传入

        for f in os.listdir(directory):
            if f.endswith("_emg.txt"):
                filepath = os.path.join(directory, f)
                if os.path.getsize(filepath) > 0:
                    self.files.append(filepath)
                    self.labels.append(f.split('_')[0])

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)
        self.labels = torch.from_numpy(self.labels).long()

        self.num_classes=len(np.unique(self.labels))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        filepath = self.files[idx]
        emg_signals = np.loadtxt(filepath)
        if emg_signals.shape[0] < self.seq_length:
            pad = np.zeros((self.seq_length - emg_signals.shape[0], emg_signals.shape[1]))
            emg_signals = np.vstack([emg_signals, pad])
        elif emg_signals.shape[0] > self.seq_length:
            emg_signals = emg_signals[:self.seq_length, :]

        if self.scaler:
            emg_signals = self.scaler.transform(emg_signals)  # 使用预先拟合的Scaler进行转换

        emg_signals = torch.from_numpy(emg_signals).float()
        return emg_signals, self.labels[idx]

    def get_num_classes(self):
		    return self.num_classes



# # Assuming 'path_to_your_directory' is the path to your data
# dataset = EMGDataset('path_to_your_directory')
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import os

class IMUDataset(Dataset):
    def __init__(self, directory, seq_length=790, scaler=None):
        self.directory = directory
        self.seq_length = seq_length
        self.files = []
        self.labels = []
        self.scaler = scaler  # Optionally use a pre-fitted scaler

        # Load files and labels based on the file naming convention
        for f in os.listdir(directory):
            if f.endswith("_imu.txt"):  # Ensure to load only IMU files
                filepath = os.path.join(directory, f)
                if os.path.getsize(filepath) > 0:  # Check if file is not empty
                    self.files.append(filepath)
                    label = f.split('_')[0]  # Assuming label is before the first underscore
                    self.labels.append(label)

        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)
        self.labels = torch.from_numpy(self.labels).long()

        self.num_classes = len(np.unique(self.labels))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        filepath = self.files[idx]
        imu_data = np.loadtxt(filepath)
        
        # Handle sequence length discrepancies
        if imu_data.shape[0] < self.seq_length:
            pad = np.zeros((self.seq_length - imu_data.shape[0], imu_data.shape[1]))
            imu_data = np.vstack([imu_data, pad])
        elif imu_data.shape[0] > self.seq_length:
            imu_data = imu_data[:self.seq_length, :]
        
        # Apply scaling if a scaler is provided
        if self.scaler:
            imu_data = self.scaler.transform(imu_data)  # Transform data using the pre-fitted scaler

        imu_data = torch.from_numpy(imu_data).float()
        return imu_data, self.labels[idx]

    def get_num_classes(self):
        return self.num_classes


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

class MyDataset(Dataset):
    def __init__(self, emg_directory, imu_directory, seq_length=790, scaler=None):
        self.seq_length = seq_length
        self.scaler = scaler  # Optionally use a pre-fitted scaler

        # Load files and labels from both directories
        self.emg_files, self.imu_files, self.labels = self.load_files(emg_directory, imu_directory)

        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)
        self.labels = torch.from_numpy(self.labels).long()

    def load_files(self, emg_directory, imu_directory):
        emg_files = []
        imu_files = []
        labels = []
        
        # Assuming same filenames for corresponding EMG and IMU files except extensions
        for f in os.listdir(emg_directory):
            if f.endswith("_emg.txt"):
                emg_filepath = os.path.join(emg_directory, f)
                imu_filepath = os.path.join(imu_directory, f.replace("_emg.txt", "_imu.txt"))
                
                if os.path.exists(imu_filepath) and os.path.getsize(emg_filepath) > 0 and os.path.getsize(imu_filepath) > 0:
                    emg_files.append(emg_filepath)
                    imu_files.append(imu_filepath)
                    labels.append(f.split('_')[0])
                    
        return emg_files, imu_files, labels

    def __len__(self):
        return len(self.emg_files)

    def __getitem__(self, idx):
        emg_filepath = self.emg_files[idx]
        imu_filepath = self.imu_files[idx]
        
        # Load data
        emg_data = np.loadtxt(emg_filepath)
        imu_data = np.loadtxt(imu_filepath)
        
        # Handle sequence length discrepancies for both data types
        if emg_data.shape[0] < self.seq_length:
            emg_data = np.vstack([emg_data, np.zeros((self.seq_length - emg_data.shape[0], emg_data.shape[1]))])
        else:
            emg_data = emg_data[:self.seq_length, :]
        
        if imu_data.shape[0] < self.seq_length:
            imu_data = np.vstack([imu_data, np.zeros((self.seq_length - imu_data.shape[0], imu_data.shape[1]))])
        else:
            imu_data = imu_data[:self.seq_length, :]
        
        # Concatenate EMG and IMU data along feature axis
        fused_data = np.concatenate([emg_data, imu_data], axis=1)
        
        if self.scaler:
            fused_data = self.scaler.transform(fused_data)  # Transform data using the pre-fitted scaler
        
        fused_data = torch.from_numpy(fused_data).float()
        return fused_data, self.labels[idx]


## 计算sacler

In [ ]:
import numpy as np
import os

def incremental_mean_std(directory, seq_length=790):
    mean = np.zeros((seq_length, 8))  # 假设每个样本有8个特征
    M2 = np.zeros((seq_length, 8))
    n = 0
    for filename in os.listdir(directory):
        if filename.endswith("_emg.txt"):
            filepath = os.path.join(directory, filename)
            if os.path.getsize(filepath) > 0:
                data = np.loadtxt(filepath)
                if data.size == 0:  # 检查数据是否为空
                    continue
                if data.shape[0] < seq_length:
                    # 填充不足的数据
                    pad = np.zeros((seq_length - data.shape[0], data.shape[1]))
                    data = np.vstack([data, pad])
                elif data.shape[0] > seq_length:
                    # 截断超出的数据
                    data = data[:seq_length, :]
                n += 1
                delta = data - mean
                mean += delta / n
                M2 += delta * (data - mean)

    std = np.sqrt(M2 / n) if n > 1 else np.sqrt(M2)
    return mean, std

# directory = 'emg_data'
# mean, std = incremental_mean_std(directory)

# 用得到的 mean 和 std 来创建一个 StandardScaler-like 的类
class CustomScaler:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, X):
        return (X - self.mean) / self.std


In [ ]:
directory = 'emg_data'
mean, std = incremental_mean_std(directory)
scaler = CustomScaler(mean, std)

In [6]:
import pickle

# 从文件加载scaler对象
# filepath = 'drive/MyDrive/scaler.pkl'
filepath = 'scaler.pkl'
with open(filepath, 'rb') as f:
    loaded_scaler = pickle.load(f)

    scaler = loaded_scaler

/usr/local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# 使用加载的scaler来转换数据
# transformed_data = loaded_scaler.transform(new_data)

In [ ]:
dataset = EMGDataset(directory=directory, scaler=scaler)

In [ ]:
directory='emg_data'
# scaler = compute_statistics(directory)

dataset = EMGDataset(directory=directory, scaler=scaler)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## 确定seq_length
确定seq_length

In [ ]:
import numpy as np
import os

lengths0 = []
lengths1 = []
def analyze_emg_lengths(directory):

    for filename in os.listdir(directory):
        if filename.endswith("_emg.txt"):
            filepath = os.path.join(directory, filename)
            try:
                emg_signals = np.loadtxt(filepath)
                if emg_signals.shape[0]!=0:
                  lengths0.append(emg_signals.shape[0])
                  lengths1.append(emg_signals.shape[1])

            except:
                # Handling empty or corrupted files
                print(f"Could not load data from {filename}")
    return lengths0, lengths1

# Assuming 'path_to_your_directory' is the path to your data
# directory = 'emg_data'
# lengths = analyze_emg_lengths(directory)

# # Calculate statistics
# if lengths0:
#     min_length0 = np.min(lengths0)
#     max_length0 = np.max(lengths0)
#     avg_length0 = np.mean(lengths0)
#     median_length0 = np.median(lengths0)

#     min_length1 = np.min(lengths1)
#     max_length1 = np.max(lengths1)
#     avg_length1 = np.mean(lengths1)
#     median_length1 = np.median(lengths1)

#     print(f"Minimum length: {min_length0}, {min_length1}")
#     print(f"Maximum length: {max_length0}， {max_length1}")
#     print(f"Average length: {avg_length0:.2f}, {avg_length1:.2f}")
#     print(f"Median length: {median_length0}, {median_length1}")
# else:
#     print("No data available to analyze.")


## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [ ]:
# scaler=scaler
scaler=None

In [7]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

def emg_collate_batch(batch):
    data, labels = zip(*batch)
    return torch.stack(data), torch.tensor(labels)

def get_emg_dataloader(data_dir, batch_size, n_workers):
    dataset = EMGDataset(directory=data_dir, scaler=scaler)
    num_classes = dataset.get_num_classes()
    # 分割数据集为训练集和验证集
    train_len = int(0.9 * len(dataset))
    lengths = [train_len, len(dataset) - train_len]
    train_set, valid_set = random_split(dataset, lengths)

    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=emg_collate_batch,
    )
    valid_loader = DataLoader(
        valid_set,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=emg_collate_batch,
    )

    return train_loader, valid_loader, num_classes


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [8]:
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, num_classes, input_dim=8, d_model=224, ff_dim=256, num_heads=2, dropout=0.1):
        super(TransformerModel, self).__init__()
        # Project the dimension of features from that of input into an enhanced feature space
        self.prenet = nn.Linear(input_dim, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=d_model*2, nhead=num_heads, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.prenet(x)  # Apply prenet
        x = self.encoder(x)  # Transformer encoder
        x = x.transpose(1, 2)  # Change (batch, seq_len, features) to (batch, features, seq_len)
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x


# Assuming 8 features per timestep and the sequence length is the same for all samples
# model = TransformerModel(input_dim=8, num_heads=2, ff_dim=256, num_classes=len(np.unique(dataset.labels)))


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [9]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [10]:
import torch

def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    # 解包批数据
    data, labels = batch
    # 将数据和标签转移到指定设备（例如 GPU）
    data = data.to(device)
    labels = labels.to(device)

    # 通过模型进行前向传播得到输出
    outs = model(data)

    # 计算损失
    loss = criterion(outs, labels)

    # 计算预测的类别（最高得分的类别）
    preds = outs.argmax(1)
    # 计算准确率
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [11]:
from tqdm import tqdm
import torch

def valid(dataloader, model, criterion, device):
    """Validate on validation set."""
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit="sample")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)


# Main function

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader

# from get_emg_dataloader import get_emg_dataloader
# from your_model_file import TransformerModel, model_fn, valid

def parse_args():
    """arguments"""
    config = {
        # "data_dir": "./emg_data",
        "data_dir": "/tmp/dataset/emg_data/emg_data",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 2,
        "valid_steps": 500,
        "warmup_steps": 500,
        "save_steps": 2500,
        "total_steps": 7500,
        "pretrained_path": None,  # 可以设置为预先训练好的模型路径
    }
    return config

def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
    pretrained_path=None,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, num_classes = get_emg_dataloader(data_dir, batch_size, n_workers)
    print(f"[Info]: Finish loading data!", flush=True)

    model = TransformerModel(num_classes=num_classes).to(device)
    if pretrained_path:
        model.load_state_dict(torch.load(pretrained_path, map_location=device))
        print("[Info]: Pretrained model loaded!")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        try:
            batch = next(iter(train_loader))
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        pbar.update()
        pbar.set_postfix(
            loss=f"{loss.item():.2f}",
            accuracy=f"{accuracy:.2f}",
            step=step + 1,
        )

        if (step + 1) % valid_steps == 0:
            pbar.close()
            valid_accuracy = valid(valid_loader, model, criterion, device)
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            print(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()

if __name__ == "__main__":
    main(**parse_args())



[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 500/500 [03:28<00:00,  2.40 step/s, accuracy=0.00, loss=5.89, step=500]
Valid: 100% 7616/7622 [00:16<00:00, 469.49sample/s, accuracy=0.01, loss=5.97]
Train: 100% 500/500 [03:18<00:00,  2.52 step/s, accuracy=0.03, loss=6.11, step=1000]
Valid: 100% 7616/7622 [00:13<00:00, 566.67sample/s, accuracy=0.01, loss=6.09]
Train: 100% 500/500 [03:16<00:00,  2.55 step/s, accuracy=0.00, loss=5.74, step=1500]
Valid: 100% 7616/7622 [00:13<00:00, 561.08sample/s, accuracy=0.01, loss=6.61]
Train: 100% 500/500 [03:14<00:00,  2.57 step/s, accuracy=0.00, loss=6.02, step=2000]
Valid: 100% 7616/7622 [00:13<00:00, 561.59sample/s, accuracy=0.00, loss=6.89]
Train: 100% 500/500 [03:13<00:00,  2.59 step/s, accuracy=0.00, loss=6.13, step=2500]
Valid: 100% 7616/7622 [00:14<00:00, 517.34sample/s, accuracy=0.00, loss=7.01]
Train:   0% 0/500 [00:00<?, ? step/s]

Step 2500, best model saved. (accuracy=0.0100)


Train: 100% 500/500 [03:12<00:00,  2.60 step/s, accuracy=0.03, loss=5.91, step=3000]
Valid: 100% 7616/7622 [00:13<00:00, 562.61sample/s, accuracy=0.00, loss=7.33]
Train: 100% 500/500 [03:11<00:00,  2.61 step/s, accuracy=0.00, loss=6.32, step=3500]
Valid: 100% 7616/7622 [00:13<00:00, 565.02sample/s, accuracy=0.01, loss=6.08]
Train: 100% 500/500 [03:11<00:00,  2.61 step/s, accuracy=0.00, loss=6.25, step=4000]
Valid: 100% 7616/7622 [00:13<00:00, 563.38sample/s, accuracy=0.00, loss=7.58]
Train: 100% 500/500 [03:11<00:00,  2.61 step/s, accuracy=0.03, loss=5.43, step=4500]
Valid: 100% 7616/7622 [00:13<00:00, 559.72sample/s, accuracy=0.01, loss=6.63]
Train: 100% 500/500 [03:10<00:00,  2.63 step/s, accuracy=0.06, loss=5.50, step=5000]
Valid: 100% 7616/7622 [00:13<00:00, 564.61sample/s, accuracy=0.01, loss=6.76]
Train:   0% 0/500 [00:00<?, ? step/s]

Step 5000, best model saved. (accuracy=0.0100)


Train: 100% 500/500 [03:11<00:00,  2.61 step/s, accuracy=0.00, loss=5.49, step=5500]
Valid: 100% 7616/7622 [00:13<00:00, 561.07sample/s, accuracy=0.01, loss=6.13]
Train:   5% 27/500 [00:10<02:59,  2.63 step/s, accuracy=0.03, loss=5.95, step=5527]

# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]